# 🧑‍🍳 Cooking up flavorful SWOTs with the kernel

## 🔥 Get a kernel ready

In [2]:
!pip install semantic_kernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 3.6 MB/s eta 0:00:004.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 10.5 MB/s eta 0:00:00
  Using cached jsonschema_specifications-2023.7.1-py3-none-any.whl (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 5.2 MB/s eta 0:00:00
  Installing build d

In [4]:
import os  # 추가
import semantic_kernel as sk

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    # api_key, org_id = sk.openai_settings_from_dot_env()
    # kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    api_key = os.environ.get("OPENAI_API_KEY")
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key))

print("A kernel is now ready.")   

A kernel is now ready.


# Let's make a semantic function

In [15]:
sk_prompt = """
{{$input}}

Summarize the content above in less than 140 characters in Korean.
"""
summary_function = kernel.create_semantic_function(prompt_template = sk_prompt,
                                                    description="Summarizes the input to length of an old tweet.",
                                                    max_tokens=200,
                                                    temperature=0.1,
                                                    top_p=0.5)       
print("A semantic function for summarization has been registered.");

A semantic function for summarization has been registered.


In [16]:
sk_input = """
Let me illustrate an example. Many weekends, I drive a few minutes from my house to a local pizza store to buy 
a slice of Hawaiian pizza from the gentleman that owns this pizza store. And his pizza is great, but he always 
has a lot of cold pizzas sitting around, and every weekend some different flavor of pizza is out of stock. 
But when I watch him operate his store, I get excited, because by selling pizza, he is generating data. 
And this is data that he can take advantage of if he had access to AI.

AI systems are good at spotting patterns when given access to the right data, and perhaps an AI system could spot 
if Mediterranean pizzas sell really well on a Friday night, maybe it could suggest to him to make more of it on a 
Friday afternoon. Now you might say to me, "Hey, Andrew, this is a small pizza store. What's the big deal?" And I 
say, to the gentleman that owns this pizza store, something that could help him improve his revenues by a few 
thousand dollars a year, that will be a huge deal to him.
""";
# Text source: https://www.ted.com/talks/andrew_ng_how_ai_could_empower_any_business/transcript

summary_result = await kernel.run_async(summary_function, input_str=sk_input)

display(Markdown("### ✨ " + str(summary_result)))

### ✨ 피자 가게 주인이 AI를 활용하면 매출을 높일 수 있다. AI는 데이터를 분석하여 패턴을 찾아주기 때문이다.

In [17]:
summary_result = summary_function(sk_input)

display(Markdown("### ✨ " + str(summary_result)))

### ✨ 피자 가게 주인이 인공지능을 활용하면 매출을 높일 수 있다. AI는 데이터를 분석하여 패턴을 찾아 추천할 수 있다.

# Native functions

In [18]:
from semantic_kernel.skill_definition import (
    sk_function,
    sk_function_context_parameter,
)

class ExoticLanguagePlugin:
    def word_to_pig_latin(self, word):
        vowels = "AEIOUaeiou"
        if word[0] in vowels:
            return word + "way"
        for idx, letter in enumerate(word):
            if letter in vowels:
                break
        else:
            return word + "ay"
        return word[idx:] + word[:idx] + "ay"
    @sk_function(
        description="Takes text and converts it to pig latin",
        name="pig_latin",
        input_description="The text to convert to pig latin",
    )
    def pig_latin(self, sentence:str) -> str:
        words = sentence.split()
        pig_latin_words = []
        for word in words:
            pig_latin_words.append(self.word_to_pig_latin(word))
        return ' '.join(pig_latin_words)

exotic_language_plugin = kernel.import_skill(ExoticLanguagePlugin(), skill_name="exotic_language_plugin")
pig_latin_function = exotic_language_plugin["pig_latin"]

print("this is kind of not going to feel awesome but know this is a big deal")


this is kind of not going to feel awesome but know this is a big deal


In [19]:
final_result = await kernel.run_async(summary_function, pig_latin_function, input_str=sk_input) 

display(Markdown("### ✨ " + str(final_result)))

### ✨ 피자ay 가게ay 주인이ay 인공지능을ay 활용하면ay 매출을ay 높일ay 수ay 있다.ay AI는way 데이터를ay 활용하여ay 매출ay 패턴을ay 파악하고ay 추천할ay 수ay 있다.ay


[![](./assets/andrew_ng.jpg)](assets/thepizzastore720.mp4)
​

1. **Strengths**
    - What unique recipes or ingredients does the pizza shop use?
    - What are the skills and experience of the staff?
    - Does the pizza shop have a strong reputation in the local area?
    - Are there any unique features of the shop or its location that attract customers?
2. **Weaknesses**
    - What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)
    - Are there financial constraints that limit growth or improvements?
    - Are there any gaps in the product offering?
    - Are there customer complaints or negative reviews that need to be addressed?
3. **Opportunities**
    - Is there potential for new products or services (e.g., catering, delivery)?
    - Are there under-served customer segments or market areas?
    - Can new technologies or systems enhance the business operations?
    - Are there partnerships or local events that can be leveraged for marketing?
4. **Threats**
    - Who are the major competitors and what are they offering?
    - Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?
    - Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?
    - Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?

| **Strengths**                                     | **Weaknesses**                                               |
| --- | --- |
| Unique garlic pizza recipe that wins top awards  | High staff turnover                                          |
| Owner trained in Sicily at some of the best pizzerias                          | Floods in the area damaged the seating areas that are in need of repair  |
| Strong local reputation                           | Absence of popular calzones from menu                        |
| Prime location on university campus               | Negative reviews from younger demographic for lack of hip ingredients |

Meanwhile there's money being left on the table (their opportunities) with calamities (their threats) waiting in the wings to possibly happen and may knock their pizza shop out cold.

### 🔖 Opportunities and Threats

| **Opportunities**                                 | **Threats**                                                  |
| --- | ---|
| Untapped catering potential                       | Rising competition from cheaper pizza businesses nearby |
| Growing local tech startup community              | There's nearby street construction that will impact foot traffic |
| Unexplored online presence and order capabilities | Rising cost of cheese                                        |
| Upcoming annual food fair                         | No immediate local regulatory changes but it's election season |

## 🤷 What does this have to do with LLMs and Semantic Kernel?

In [12]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key = os.environ.get("OPENAI_API_KEY")
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key))
    # api_key, org_id = sk.openai_settings_from_dot_env()
    # kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
print("Made a kernel!")

Made a kernel!


In [24]:
swot_interview= """
1. **Strengths**
    - What unique recipes or ingredients does the pizza shop use?
    - What are the skills and experience of the staff?
    - Does the pizza shop have a strong reputation in the local area?
    - Are there any unique features of the shop or its location that attract customers?
2. **Weaknesses**
    - What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)
    - Are there financial constraints that limit growth or improvements?
    - Are there any gaps in the product offering?
    - Are there customer complaints or negative reviews that need to be addressed?
3. **Opportunities**
    - Is there potential for new products or services (e.g., catering, delivery)?
    - Are there under-served customer segments or market areas?
    - Can new technologies or systems enhance the business operations?
    - Are there partnerships or local events that can be leveraged for marketing?
4. **Threats**
    - Who are the major competitors and what are they offering?
    - Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?
    - Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?
    - Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"""


sk_prompt = """
{{$input}}

Convert the analysis provided above to the business domain of {{$domain}}. Reply in Korean
"""
shift_domain_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Translate an idea to another domain.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)
my_context = kernel.create_new_context()

my_context['input'] = swot_interview
# my_context['domain'] = "construction management"
my_context['domain'] = "software development"

result = await kernel.run_async(shift_domain_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']}\n"+ str(result)))


### ✨ Shift the SWOT interview questions to the world of software development
위의 분석을 소프트웨어 개발 비즈니스 도메인으로 변환하면 다음과 같습니다.

1. **강점**
    - 소프트웨어 개발에서 고유한 기술이나 알고리즘을 사용하는가?
    - 개발자들의 기술과 경험이 충분한가?
    - 로컬 지역에서 소프트웨어 개발 회사의 평판이 좋은가?
    - 회사나 개발자의 위치나 특징이 고객을 끌어들이는가?
2. **약점**
    - 소프트웨어 개발에서 운영상의 문제가 있는가? (예: 느린 서비스, 높은 인력 이동률)
    - 성장이나 개선을 제한하는 재정적 제약이 있는가?
    - 제품 제공에서 빈틈이 있는가?
    - 고객 불만이나 부정적인 리뷰가 있는가?
3. **기회**
    - 새로운 제품이나 서비스 (예: 새로운 앱, 클라우드 서비스)를 출시할 수 있는 기회가 있는가?
    - 미처 다가오지 않은 고객 세그먼트나 시장 영역이 있는가?
    - 새로운 기술이나 시스템이 비즈니스 운영을 개선할 수 있는가?
    - 마케팅에 활용할 수 있는 파트너십이나 로컬 이벤트가 있는가?
4. **위협**
    - 주요 경쟁 업체가 누구이며 그들이 제공하는 것은 무엇인가?
    - 지역 내 변화 (예: 건설, 인근 비즈니스 폐쇄)로 인한 부정적인 영향이 있는가?
    - 산업 경제나 동향이 비즈니스에 부정적인 영향을 미칠 가능성이 있는가? (예: 증가하는 개발 비용)
    - 규제나 법률의 변화로 인해 위험이 있는가? (예: 개인정보 보호, 저작권)

In [25]:
sk_prompt = """
{{$input}}

Rewrite the text above to be understood by a {{$level}}. Rewrite in Korean.
"""
shift_reading_level_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Change the reading level of a given text.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)

my_context['input'] = swot_interview
my_context['domain'] = "software development"
my_context["level"] = "cto"

result = await kernel.run_async(shift_domain_function, shift_reading_level_function, input_context=my_context)

display(Markdown(f"### ✨ Shift the SWOT interview questions to the world of {my_context['domain']} at the level of {my_context['level']}\n"+ str(result)))

### ✨ Shift the SWOT interview questions to the world of software development at the level of cto
위의 분석을 CTO가 이해할 수 있도록 소프트웨어 개발 비즈니스 도메인으로 다음과 같이 재구성합니다.

1. **강점**
    - 기술적 우위: 소프트웨어 개발에서 고유한 기술이나 알고리즘을 사용하는가?
    - 인력 우수성: 개발자들의 기술과 경험이 충분한가?
    - 평판: 로컬 지역에서 소프트웨어 개발 회사의 평판이 좋은가?
    - 독특한 제품/서비스: 고객을 유치하는 독특한 제품이나 서비스가 있는가?
2. **약점**
    - 운영상 문제: 소프트웨어 개발에서 운영상의 문제가 있는가? (예: 느린 서비스, 높은 인력 유동성)
    - 재정적 제약: 성장이나 개선을 제한하는 재정적 제약이 있는가?
    - 제품 라인업 빈틈: 제품 라인업에 빈틈이 있는가?
    - 고객 불만/부정적 리뷰: 고객 불만이나 부정적인 리뷰가 있는가?
3. **기회**
    - 새로운 제품/서비스 출시: 새로운 제품이나 서비스 (예: 컨설팅, 클라우드 서비스)를 출시할 수 있는 기회가 있는가?
    - 미처 다가오지 못한 고객 세그먼트/시장 영역: 미처 다가오지 못한 고객 세그먼트나 시장 영역이 있는가?
    - 기술/시스템 개선: 새로운 기술이나 시스템이 비즈니스 운영을 개선할 수 있는가?
    - 마케팅 파트너십/로컬 이벤트: 마케팅에 활용할 수 있는 파트너십이나 로컬 이벤트가 있는가?
4. **위협**
    - 경쟁 업체: 주요 경쟁 업체가 누구이며 어떤 제품이나 서비스를 제공하는가?
    - 지역 변화: 지역 내 변화 (예: 건설, 인근 비즈니스 폐쇄)로 인한 부정적인 영향이 있는가?
    - 산업 경제/트렌드 변화: 산업 경제나 트렌드 변화로 인해 비즈니스에 부정적인 영향을 미칠 가능성이 있는가? (예: 증가하는 개발 비용)
    - 규제/법률 변경: 규제나 법률 변경으로 인해 위험이 있는가? (예: 개인정보 보호, 저작권)

## 🔖 Reminder: All ✨ generative responses result from having the model fill in the _____.

![](./assets/completion.png)

Reminder: Download your notebook to your computer if you wish to save your work.